In [1]:
import sys
import os

sys.path.append(
    os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
)  # Adjust as needed
sys.path.append(
    os.path.abspath(os.path.join(os.getcwd(), "..", "..", "scripts"))
)  # Adjust as needed

import pandas as pd

import scripts.my_text_cleaning as tc

In [2]:
chosen_dataset = "cop26_tweets_en"
chosen_dataset = "covid_tweets_en"
chosen_dataset = "ukraine_tweets_en"
ur_df = pd.read_parquet("./../../data/raw/" + chosen_dataset + ".parquet")
# change type of 'author_id' to string
# to ensure character-by-character comparison
ur_df["author_id"] = ur_df["author_id"].astype(str)
ur_df = tc.clean_dataframe(
    ur_df,
    column_name="text",
    phrases_to_remove=["&gt;", "&lt;", "&amp;", "RT : "],
    remove_empty=False,
    normalize_hashtags=True,
    normalize_mentions=True,
    user_placeholder="user",
)
print(f"{chosen_dataset.replace('_tweets_en', '').upper()} dataset loaded.")
# print(ur_df.columns)
doc_info = pd.read_csv(
    "./../../data/processed/document_info_" + chosen_dataset + ".csv"
)[["Document", "Topic", "Representative_document", "Name"]]
doc_info.Topic = doc_info.Topic.astype(int)
print("Document-topic assignments loaded.")
# embeddings = np.load('./../../data/processed/'+chosen_dataset+'.parquet.npy')
# print(doc_info.columns)
usr_info = pd.read_csv("./../../data/raw/political_stance_merged.csv")
topic_info = pd.read_csv(
    "./../../data/processed/topic_info_" + chosen_dataset + "_with_MMR.csv"
)[["Topic", "New_Name"]]
topic_info.rename(columns={"New_Name": "Topic_Name"}, inplace=True)
print("User metadata loaded.")


UKRAINE dataset loaded.
Document-topic assignments loaded.
User metadata loaded.


In [3]:
topic_info

,Topic,Topic_Name
0,-1,-1_russian_war_ukrainian_putin_us_military_peo...
1,0,0_forces_kherson_offensive_troops_kharkiv_regi...
2,1,1_putin_west_kremlin_propaganda_twitter_war_me...
3,2,2_thread_read_good_great_thank_tweet_oh_well_t...
4,3,3_tank_destroyed_captured_artillery_brigade_uk...
...,...,...
134,133,133_passports_citizenship_passport_decree_resi...
135,134,134_poll_support_levada_polls_russians_opinion...
136,135,135_art_artist_mural_artists_exhibition_painti...
137,136,136_christmas_merry_santa_celebrate_tree_hallo...


In [4]:

doc_info = doc_info.drop(columns=["Name"]).merge(topic_info, on="Topic", how="left")

doc_info

,Document,Topic,Representative_document,Topic_Name
0,RT user: Video of a damaged Ukrainian M777 how...,3,False,3_tank_destroyed_captured_artillery_brigade_uk...
1,user Russia just accidentally manages to take ...,-1,False,-1_russian_war_ukrainian_putin_us_military_peo...
2,"RT user: For people who miss the old user, thi...",-1,False,-1_russian_war_ukrainian_putin_us_military_peo...
3,"NOW: Separatist🇷🇺 chat in Lugansk, occupied-Uk...",-1,False,-1_russian_war_ukrainian_putin_us_military_peo...
4,Location seems to be Industrial Area of Luhans...,-1,False,-1_russian_war_ukrainian_putin_us_military_peo...
...,...,...,...,...
787867,user user And you have proof Ukraine is murder...,81,False,81_genocide_committing_genocidal_ukrainians_ge...
787868,Putin 'could declare war' on May 9 in final pu...,-1,False,-1_russian_war_ukrainian_putin_us_military_peo...
787869,"Davyd Arakhamia MP: """"Unfortunately, the globa...",-1,False,-1_russian_war_ukrainian_putin_us_military_peo...
787870,'The fact is Russia have taken two humanitaria...,76,False,76_killed_died_commander_lieutenant_colonel_ri...


In [5]:

# print(usr_info.columns)  # merge on 'username'
# merge ur_df with doc_info on 'Cleantext' and 'Document'
# left join to keep all rows in ur_df
# check that length is the same before and after merge
merged_df = ur_df.merge(doc_info, left_on="Cleantext", right_on="Document", how="left")
assert len(merged_df) == len(ur_df), (
    "differing length before and after merge, duplicates in right"
)
print("Tweet-topic assignment merged.")

Tweet-topic assignment merged.


In [6]:
merged_df

,id,author_id,created_at,lang,in_reply_to_user_id,conversation_id,text,reply_settings,possibly_sensitive,retweeted_id,...,like_count,quote_count,username,individual_or_org,category,Cleantext,Document,Topic,Representative_document,Topic_Name
0,1546962210194325505,1082604507454939137,2022-07-12T20:58:26.000Z,en,,1546962210194325505,RT @RALee85: Video of a damaged Ukrainian M777...,everyone,False,1546945222738907137,...,0,0,Archer83Able,Organization,Media,RT user: Video of a damaged Ukrainian M777 how...,RT user: Video of a damaged Ukrainian M777 how...,3,False,3_tank_destroyed_captured_artillery_brigade_uk...
1,1546962286513881090,3028079651,2022-07-12T20:58:44.000Z,en,1462548977367359490,1546960688899395585,@KyivIndependent Russia just accidentally mana...,everyone,False,,...,63,0,mdfzeh,Organization,Other,user Russia just accidentally manages to take ...,user Russia just accidentally manages to take ...,-1,False,-1_russian_war_ukrainian_putin_us_military_peo...
2,1546962379870801920,60698597,2022-07-12T20:59:07.000Z,en,,1546962379870801920,RT @aaronjmate: For people who miss the old @d...,everyone,False,1546906745548726272,...,0,0,Consortiumnews,Organization,Media,"RT user: For people who miss the old user, thi...","RT user: For people who miss the old user, thi...",-1,False,-1_russian_war_ukrainian_putin_us_military_peo...
3,1546962441573203971,888449203688677377,2022-07-12T20:59:21.000Z,en,,1546962441573203971,"NOW: Separatist🇷🇺 chat in #Lugansk, occupied-U...",everyone,False,,...,443,4,officejjsmart,Individual,Media,"NOW: Separatist🇷🇺 chat in Lugansk, occupied-Uk...","NOW: Separatist🇷🇺 chat in Lugansk, occupied-Uk...",-1,False,-1_russian_war_ukrainian_putin_us_military_peo...
4,1546962711879319552,556214449,2022-07-12T21:00:26.000Z,en,556214449,1546961828277788672,Location seems to be Industrial Area of Luhans...,everyone,False,,...,15,0,aldin_aba,Individual,Private Individual,Location seems to be Industrial Area of Luhans...,Location seems to be Industrial Area of Luhans...,-1,False,-1_russian_war_ukrainian_putin_us_military_peo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916950,1520037135373455361,813738086282694656,2022-04-29T13:47:48.000Z,en,2780341250,1520002933991124993,@World_Socialism @One0fTheMany And you have pr...,everyone,False,,...,0,0,56blackcat,Individual,Private Individual,user user And you have proof Ukraine is murder...,user user And you have proof Ukraine is murder...,81,False,81_genocide_committing_genocidal_ukrainians_ge...
916951,1520037194106392576,17895820,2022-04-29T13:48:02.000Z,en,,1520037194106392576,Putin 'could declare war' on May 9 in final pu...,everyone,False,,...,5,0,Daily_Express,Organization,Media,Putin 'could declare war' on May 9 in final pu...,Putin 'could declare war' on May 9 in final pu...,-1,False,-1_russian_war_ukrainian_putin_us_military_peo...
916952,1520037229930115077,1497351174487298052,2022-04-29T13:48:11.000Z,en,,1520037229930115077,"Davyd Arakhamia MP:\n\n""""Unfortunately, the gl...",everyone,False,,...,1,0,Now_in_Ukraine,Organization,International Organization / NGO,"Davyd Arakhamia MP: """"Unfortunately, the globa...","Davyd Arakhamia MP: """"Unfortunately, the globa...",-1,False,-1_russian_war_ukrainian_putin_us_military_peo...
916953,1520037259596247051,1339166129110065152,2022-04-29T13:48:18.000Z,en,,1520037259596247051,'The fact is Russia have taken two humanitaria...,everyone,False,,...,16,0,GBNEWS,Organization,Media,'The fact is Russia have taken two humanitaria...,'The fact is Russia have taken two humanitaria...,76,False,76_killed_died_commander_lieutenant_colonel_ri...


In [7]:
# save merged_df to csv
merged_df.to_csv(
    "./../../data/processed/merged_tweet_topic_" + chosen_dataset + ".csv", index=False
)